In [1]:
import os
os.chdir("..")

import torch
import torch.nn as nn
import numpy as np
from torch import optim
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchmetrics import MeanAbsolutePercentageError
from torch.utils.data import DataLoader, TensorDataset

from data.dataloader import dl_from_numpy, dataloader_info
from models.predictor import GRU
from utils.utils import load_yaml_config, instantiate_from_config

/home/harim/Desktop/pyproject/Project_TS_Generation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load configurations
configs = load_yaml_config("configs/stock_prediction.yaml")
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load dataset info
dl_info_train = dataloader_info(configs)
dl_info_test = dataloader_info(configs, train=False)
dataset = dl_info_train['dataset']
seq_length, feature_dim = dataset.window, dataset.feature_dim
batch_size = configs["dataloader"]["batch_size"]


# dataset
test_data_norm = torch.from_numpy(np.load(os.path.join(dataset.dir, f"stock_ground_truth_data_{seq_length}_test.npy"))).to(device)
test_mean = torch.from_numpy(np.load(os.path.join(dataset.dir, f"stock_ground_truth_mean_{seq_length}_test.npy"))).to(device)
test_std = torch.from_numpy(np.load(os.path.join(dataset.dir, f"stock_ground_truth_std_{seq_length}_test.npy"))).to(device)
test_dataset = TensorDataset(test_data_norm, test_mean, test_std)
# fake_dataset = torch.from_numpy(np.load(os.path.join(dataset.dir, f"ddpm_fake_stock.npy"))).to(device)
# ori_fake_dataset = TensorDataset(dl_info_train["dataset"], fake_dataset)

# load dataloader
ori_dl = dl_info_train["dataloader"]
fake_dl = dl_from_numpy(os.path.join(dataset.dir, f"ddpm_fake_stock.npy"), batch_size=batch_size)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# ori_fake_dl = DataLoader(ori_fake_dataset, batch_size=batch_size, shuffle=False)

lossfn = nn.L1Loss()
lr = 0.001


In [3]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=100, description=""):
    model.train()
    with tqdm(range(num_epochs), total=num_epochs) as pbar:
        for e in pbar:
            for data in dataloader:
                x_train = data[:,:-1,:].float().to(device)
                y_train = data[:,-1:,0].float().to(device)
                optimizer.zero_grad()
                outputs = model(x_train)
                loss = criterion(outputs, y_train)
                loss.backward()
                optimizer.step()
            pbar.set_description(f"{description} loss: {loss.item():.6f}")
    

In [4]:
def evaluate_model(model, dataloader):
    model.eval()
    
    # define loss for comparison
    l1loss = nn.L1Loss()
    l2loss = nn.MSELoss()
    mapeloss = MeanAbsolutePercentageError().to(device)
    
    total_l1 = 0
    total_l2 = 0
    total_mape = 0

    predictions, true_vals = [], []
    with torch.no_grad():
        for data, mean, std in dataloader:
            x_test = data[:, :(seq_length - 1), :].float().to(device)
            y_test = data[:, (seq_length - 1):, :1].float().to(device)
            mean = mean[:, :, :1].float().to(device)
            std = std[:, :, :1].float().to(device)
            y_pred = model(x_test).view(-1,1,1)
            
            total_l1 += l1loss(y_pred, y_test) * len(data)
            total_l2 += l2loss(y_pred, y_test) * len(data)
            total_mape += mapeloss(y_pred, y_test).item()

            y_test_unnorm = y_test * std + mean
            y_pred_unnorm = y_pred * std + mean

            predictions.append(y_pred_unnorm.cpu().numpy())
            true_vals.append(y_test_unnorm.cpu().numpy())

    n_data = len(dataloader.dataset)
    total_l1 /= n_data
    total_l2 /= n_data
    total_mape /= n_data
    
    predictions = np.concatenate(predictions).squeeze()
    true_vals = np.concatenate(true_vals).squeeze()
    # mape_loss = mapeloss(torch.tensor(predictions), torch.tensor(true_vals)).item()
    
    return total_l1, total_l2, total_mape, predictions, true_vals

In [5]:
# Initialize Diffusion_TS Model
diffusion_adversarial = instantiate_from_config(configs['model']).to(device)
diffusion_adversarial.load_state_dict(torch.load("check_points/stock_24/model_50000.pth"))

<All keys matched successfully>

In [6]:
# Train on original + adv_synthetic data
synadv_ori_l1_list = []
synadv_ori_l2_list = []
synadv_ori_mape_list = []
ori_dataset = dl_info_train["dataset"].data.to(device)

e = 2000
for _ in range(5):
    model_ori = GRU(input_dim=feature_dim, hidden_dim=50, num_layers=2).to(device)
    optimizer_ori = optim.Adam(model_ori.parameters(), lr=lr)
    
    # train original model on original data
    train_model(model_ori, ori_dl, lossfn, optimizer_ori, num_epochs=e, description="Original")
    
    # make adv_data on trained original model
    adv_data = diffusion_adversarial.generate_adversarial(ori_dataset, predictor=model_ori, batch_size=128, num_timesteps=10)
    adv_data = adv_data.to(device)
    ori_adv_dl = DataLoader(adv_data, batch_size=128, shuffle=True)
    
    # retrain original model on synthetic adversarial data
    train_model(model_ori, ori_adv_dl, lossfn, optimizer_ori, num_epochs=e, description="SynAdv")

    synadv_ori_l1, synadv_ori_l2, synadv_ori_mape, synadv_ori_pred_y, _ = evaluate_model(model_ori, test_dl)
    synadv_ori_l1_list.append(synadv_ori_l1.item())
    synadv_ori_l2_list.append(synadv_ori_l2.item())
    synadv_ori_mape_list.append(synadv_ori_mape)
    print(f"Adv_synthetic : L1 loss: {synadv_ori_l1:0.5f} \t L2 Loss : {synadv_ori_l2:0.5f} \t MAPE loss : {synadv_ori_mape:0.5f} ")


  0%|          | 0/27 [00:00<?, ?it/s]/home/harim/Desktop/pyproject/Project_TS_Generation/.venv/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
SynAdv loss: 0.016951: 100%|██████████| 2000/2000 [00:45<00:00, 44.10it/s]


Adv_synthetic : L1 loss: 0.59515 	 L2 Loss : 0.66888 	 MAPE loss : 0.01390 


SynAdv loss: 0.019030: 100%|██████████| 2000/2000 [00:46<00:00, 42.77it/s]


Adv_synthetic : L1 loss: 0.59762 	 L2 Loss : 0.68249 	 MAPE loss : 0.01258 


SynAdv loss: 0.020690: 100%|██████████| 2000/2000 [00:47<00:00, 42.50it/s]


Adv_synthetic : L1 loss: 0.59301 	 L2 Loss : 0.67077 	 MAPE loss : 0.01461 


SynAdv loss: 0.021582: 100%|██████████| 2000/2000 [00:45<00:00, 44.00it/s]


Adv_synthetic : L1 loss: 0.60828 	 L2 Loss : 0.68751 	 MAPE loss : 0.01471 


SynAdv loss: 0.018578: 100%|██████████| 2000/2000 [00:45<00:00, 44.28it/s]

Adv_synthetic : L1 loss: 0.61220 	 L2 Loss : 0.69547 	 MAPE loss : 0.01543 


In [7]:
print(f"synadv_ori_l1 : \t mean : {np.mean(synadv_ori_l1_list):0.4f} \t std : {np.std(synadv_ori_l1_list):0.4f}")
print(f"synadv_ori_l2 : \t mean : {np.mean(synadv_ori_l2_list):0.4f} \t std : {np.std(synadv_ori_l2_list):0.4f}")
print(f"synadv_ori_mape : \t mean : {np.mean(synadv_ori_mape_list):0.4f} \t std : {np.std(synadv_ori_mape_list):0.4f}")


synadv_ori_l1 : 	 mean : 0.601 	 std : 0.008
synadv_ori_l2 : 	 mean : 0.681 	 std : 0.010
synadv_ori_mape : 	 mean : 0.014 	 std : 0.001


In [8]:
# Train on original data
ori_l1_list = []
ori_l2_list = []
ori_mape_list = []
e = 4000

for _ in range(5):
    model_ori = GRU(input_dim=feature_dim, hidden_dim=50, num_layers=2).to(device)
    optimizer_ori = optim.Adam(model_ori.parameters(), lr=lr)
    train_model(model_ori, ori_dl, lossfn, optimizer_ori, num_epochs=e, description="Original")
    
    ori_l1, ori_l2, ori_mape, ori_pred_y, true_y = evaluate_model(model_ori, test_dl)
    ori_l1_list.append(ori_l1.item())
    ori_l2_list.append(ori_l2.item())
    ori_mape_list.append(ori_mape)
    print(f"Original : L1 loss: {ori_l1:0.5f} \t L2 Loss : {ori_l2:0.5f} \t MAPE loss : {ori_mape:0.5f} ")


Original loss: 0.019538: 100%|██████████| 4000/4000 [01:34<00:00, 42.43it/s]


Original : L1 loss: 0.59797 	 L2 Loss : 0.67639 	 MAPE loss : 0.01425 


Original loss: 0.019888: 100%|██████████| 4000/4000 [01:36<00:00, 41.49it/s]


Original : L1 loss: 0.61059 	 L2 Loss : 0.71364 	 MAPE loss : 0.01540 


Original loss: 0.023438: 100%|██████████| 4000/4000 [01:50<00:00, 36.13it/s]


Original : L1 loss: 0.62171 	 L2 Loss : 0.72752 	 MAPE loss : 0.01526 


Original loss: 0.020708: 100%|██████████| 4000/4000 [01:33<00:00, 42.69it/s]


Original : L1 loss: 0.60606 	 L2 Loss : 0.71461 	 MAPE loss : 0.01411 


Original loss: 0.023456:  62%|██████▏   | 2485/4000 [00:58<00:35, 42.67it/s]

In [ ]:
print(f"ori_l1 : \t mean : {np.mean(ori_l1_list):0.4f} \t std : {np.std(ori_l1_list):0.4f}")
print(f"ori_l2 : \t mean : {np.mean(ori_l2_list):0.4f} \t std : {np.std(ori_l2_list):0.4f}")
print(f"ori_mape : \t mean : {np.mean(ori_mape_list):0.4f} \t std : {np.std(ori_mape_list):0.4f}")


In [ ]:
# Train on Synthetic data
syn_l1_list = []
syn_l2_list = []
syn_mape_list = []
e = 4000
for _ in range(5):
    model_syn = GRU(input_dim=feature_dim, hidden_dim=50, num_layers=2).to(device)
    optimizer_syn = optim.Adam(model_syn.parameters(), lr=lr)
    train_model(model_syn, fake_dl, lossfn, optimizer_syn, num_epochs=e, description="Synthetic")
    
    syn_l1, syn_l2, syn_mape, syn_pred_y, _ = evaluate_model(model_syn, test_dl)
    syn_l1_list.append(syn_l1.item())
    syn_l2_list.append(syn_l2.item())
    syn_mape_list.append(syn_mape)
    print(f"Synthetic : L1 loss: {syn_l1:0.5f} \t L2 Loss : {syn_l2:0.5f} \t MAPE loss : {syn_mape:0.5f} ")


In [ ]:
print(f"syn_l1 : \t mean : {np.mean(syn_l1_list):0.4f} \t std : {np.std(syn_l1_list):0.4f}")
print(f"syn_l2 : \t mean : {np.mean(syn_l2_list):0.4f} \t std : {np.std(syn_l2_list):0.4f}")
print(f"syn_mape : \t mean : {np.mean(syn_mape_list):0.4f} \t std : {np.std(syn_mape_list):0.4f}")


In [ ]:
def adv_train_model(model, dataloader, criterion, optimizer, epsilon=0.01, num_epochs=100, description=""):
    model.train()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    adv_data = []
    with tqdm(range(num_epochs), total=num_epochs) as pbar:
        for e in pbar:
            running_loss = 0.0
            for data in dataloader:
                x_train = data[:, :-1, :].float().to(device)
                y_train = data[:, -1:, 0].float().to(device)
                x_train.requires_grad = True
                
                # Standard training forward pass
                optimizer.zero_grad()
                outputs = model(x_train)
                loss = criterion(outputs, y_train)
                loss.backward()
                
                # Generate adversarial examples
                grad = x_train.grad.data
                x_adv = (x_train + epsilon * grad.sign()).clamp(-4, 4).detach()
                
                if e+1 == num_epochs: 
                    adv_data.append(x_adv.cpu())

                # Adversarial training forward pass
                x_adv.requires_grad = False
                outputs_adv = model(x_adv)
                loss_adv = criterion(outputs_adv, y_train)
                loss_adv.backward()
                optimizer.step()
                
                running_loss += loss_adv.item()
            
            avg_loss = running_loss / len(dataloader)
            pbar.set_description(f"{description} loss: {avg_loss:.6f}")
            
    return adv_data

In [ ]:
# Train on original + adv_original data
adv_ori_l1_list = []
adv_ori_l2_list = []
adv_ori_mape_list = []
e = 2000
epsilon = 0.1

for _ in range(5):
    model_ori = GRU(input_dim=feature_dim, hidden_dim=50, num_layers=2).to(device)
    optimizer_ori = optim.Adam(model_ori.parameters(), lr=lr)
    train_model(model_ori, ori_dl, lossfn, optimizer_ori, num_epochs=e, description="Original")
    adv_ori_data = adv_train_model(model_ori, ori_dl, lossfn, optimizer_ori, epsilon=epsilon, num_epochs=e, description="Adv_Original")

    adv_ori_l1, adv_ori_l2, adv_ori_mape, adv_ori_pred_y, _ = evaluate_model(model_ori, test_dl)
    adv_ori_l1_list.append(adv_ori_l1.item())
    adv_ori_l2_list.append(adv_ori_l2.item())
    adv_ori_mape_list.append(adv_ori_mape)
    print(f"Adv_Original : L1 loss: {adv_ori_l1:0.5f} \t L2 Loss : {adv_ori_l2:0.5f} \t MAPE loss : {adv_ori_mape:0.5f} ")


In [ ]:
print(f"adv_ori_l1 : \t mean : {np.mean(adv_ori_l1_list):0.4f} \t std : {np.std(adv_ori_l1_list):0.4f}")
print(f"adv_ori_l2 : \t mean : {np.mean(adv_ori_l2_list):0.4f} \t std : {np.std(adv_ori_l2_list):0.4f}")
print(f"adv_ori_mape : \t mean : {np.mean(adv_ori_mape_list):0.4f} \t std : {np.std(adv_ori_mape_list):0.4f}")